In [34]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import psycopg2
Base = declarative_base()

In [35]:
#Import CSVs and Creating DataFrames
housing = pd.read_csv('reduced_realty_df.csv')
housing.head()

careers_url = "web_scrape_top-100-careers.html"
careers_mega = pd.read_html(careers_url)
careers = careers_mega[2]
careers.head()

employment_data = pd.read_excel("state_M2021_dl.xlsx")
employment_data.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,17.91,28.68,39.79,18630,24720,37250,59660,82760,NaN,NaN
1,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,45.51,61.25,87.58,46900,62790,94650,127400,182160,NaN,NaN
2,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1011,Chief Executives,...,61.14,82.09,#,59190,94360,127170,170750,#,NaN,NaN
3,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,...,46.28,68.75,#,46220,62260,96270,143000,#,NaN,NaN
4,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1031,Legislators,...,*,*,*,16930,17310,17840,31300,56420,True,NaN


In [36]:
housing.columns

Index(['status', 'price', 'city', 'state', 'sold_date', 'house_size'], dtype='object')

In [37]:
#Groupby and Rename Columns
housing_by_state = housing.groupby('state').mean()
housing_by_state = housing_by_state[['price', 'house_size']]
housing_by_state.columns = ['Average Price', 'Average Square Feet']
housing_by_state['Average Price'] = round(housing_by_state['Average Price'], 2)
housing_by_state.head()

,Average Price,Average Square Feet
state,,
Connecticut,427500.45,2031.921645
Massachusetts,1110065.96,2283.397881
New Hampshire,596461.02,2269.087580
New Jersey,333490.00,1500.000000
New York,790566.40,2858.399378


In [38]:
employment_df = employment_data[['OCC_TITLE', 'A_MEAN', 'AREA_TITLE']]
employment_df.head()

,OCC_TITLE,A_MEAN,AREA_TITLE
0,All Occupations,48110,Alabama
1,Management Occupations,106210,Alabama
2,Chief Executives,150260,Alabama
3,General and Operations Managers,113350,Alabama
4,Legislators,28520,Alabama


In [39]:
# Edit Occupation Names
employment_df = employment_df.replace(to_replace='Dentists, General', value='Dentist')
employment_df = employment_df.replace(to_replace='Registered Nurses', value='Registered Nurse')
employment_df = employment_df.replace(to_replace='Pharmacists', value='Pharmacist')
employment_df = employment_df.replace(to_replace='Computer Systems Analysts', value='Computer Systems Analyst')
employment_df = employment_df.replace(to_replace='Family Medicine Physicians', value='Physician')

In [40]:
#TODO Merge Tables
    # Table 1 Jobs: Job Title, Location, Salary (Range), (Unemployment)
    # Table 2 Housing: Housing Price, Location, Rooms, Acreage, Sq Ft-age
Jobs_df = careers.merge(employment_df, left_on=1, right_on='OCC_TITLE', how='inner')
Jobs_df = Jobs_df[[1, 4, 'A_MEAN', 'AREA_TITLE']]
Jobs_df.columns = ['Occupation', 'National Unemployment Rate', 'State Mean Income', 'State']
Jobs_df.head()

,Occupation,National Unemployment Rate,State Mean Income,State
0,Dentist,0.7%,176910,Alabama
1,Dentist,0.7%,168390,Alaska
2,Dentist,0.7%,175030,Arizona
3,Dentist,0.7%,184540,Arkansas
4,Dentist,0.7%,165950,California


In [41]:
#TODO Export into PGAdmin4

In [42]:
rds_connection_string = "postgres:postgres@localhost:5432/Employment_ForHousing"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [43]:
class Housing(Base):
    __tablename__ = 'housing_by_state'
    id = Column(Integer, primary_key=True)
    Average_Price = Column(Float)
    Average_Square_Feet = Column(Float)
    state = Column(String(255))

In [44]:
class Jobs(Base):
    __tablename__ = 'jobs'
    id = Column(Integer, primary_key=True)
    Occupation = Column(String(255))
    National_Unemployment_Rate = Column(Float)
    State_Mean_Income = Column(Float)
    State = Column(String(255))

In [50]:
#Establishing the connection
conn = psycopg2.connect(
   database="Employment_ForHousing", user='postgres', password='password', host='127.0.0.1', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Droping table if already exists.
cursor.execute("DROP TABLE IF EXISTS Jobs")


#Creating table as per requirement
sql ='''CREATE TABLE Jobs(
   id INTEGER NOT NULL PRIMARY KEY,
   Occupation VARCHAR(255),
   National Unemployment Rate FLOAT,
   State Mean Income FLOAT,
   State VARCHAR(255)
)'''
cursor.execute(sql)
print("Table created successfully........")
conn.commit()
#Closing the connection
conn.close()

OperationalError: connection to server at "127.0.0.1", port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


In [51]:
#Establishing the connection
conn = psycopg2.connect(
   database="Employment_ForHousing", user='postgres', password='password', host='127.0.0.1', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Droping table if already exists.
cursor.execute("DROP TABLE IF EXISTS Housing")


#Creating table as per requirement
sql ='''CREATE TABLE Housing(
   id INTEGER NOT NULL PRIMARY KEY,
   Average Price FLOAT,
   Average Square Feet FLOAT,
   State VARCHAR(255)
)'''
cursor.execute(sql)
print("Table created successfully........")
conn.commit()
#Closing the connection
conn.close()

OperationalError: connection to server at "127.0.0.1", port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


In [52]:
# Use pandas to load DataFrame into database

housing_by_state.to_sql(name='housing_by_state', con=engine, if_exists='append', index=False)
Jobs_df.to_sql(name='jobs', con=engine, if_exists='append', index=False) 

OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/14/e3q8)